# **INSTRUCTIONS**

Note: please DO NOT change the names of the master files, config files, updaters, or the folders that they are contained in. It will break the notebook.

###**WHAT THE NOTEBOOK DOES**

1) Retrieve relevant data from the census bureau using the Census API (the API is for the ACS data profiles).

2) Insert the retrieved data into the demographics, income, and housing master sheets.

### **HOW TO RUN THE NOTEBOOK**
To use the notebook, go to "Runtime" and click "Run All". A popup will appear asking for permission to access your google drive. Give the notebook permissions for everything by connecting it to your google account.

The notebook will only edit files contained in the folder titled "SVED Economic Profile Auto-Updaters".

Notebook execution typically takes less than 45 seconds. If execution takes more than 5 minutes, or if you encounter an error, please go to "Runtime" and select "Run All" again, and the problem should resolve itself.

### **HOW TO ACCESS MASTERS**##

The masters can be accessed from the "Masters" folder. Please do not remove them. If you want them on your local machine, please download them.

### **HOW TO SET CONFIGURATION FILE**

The set the config file, edit "Census_updater_config.txt". The config file contains instructions for how to edit it. The most important thing to know is that you should ONLY change the values of the variables, and nothing else in the file (including its name).

In general, the only variable you will have to change is the year

In [30]:
import re
import csv
import json
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import numpy as np
import requests
import ast
import pandas as pd

# 1. Configuration

In [31]:
## 1.1 Give Google Colab Access to your drive. This is important. Give it access to everything.

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
## 1.2 Get Config from File

In [34]:
def get_parameters(config : str, parameter_names: list, parameter_types: list):
    """
    config -- the directory of the config file
    parameters_names -- the names of the parameters to fetch from the config file
    parameter_types -- the types of the parameters in parameters_names. Must match the length of parameter_names

    returns -- parameters, a list-like of tuples: (parameter value, parameter type)
    """
    parameters = {}
    i = config.find("%%%%%%%")
    config = config[i:-1]
    j = 0
    for param in parameter_names:
        value = ""
        i = config.find(param) + len(param) + 1
        while (config[i] != "\n"):
            if(config[i] != " " and config[i] != ":"):
                value += config[i]
            i += 1
        parameters[param] = [value, parameter_types[j]]
        j += 1
    return parameters

In [35]:
def validate_parameters(parameters):
    """
    parameters -- a list-like of tuples: (parameter value, parameter type).

    returns -- 0 if parameters are valid. -1 otherwise.
    """
    for param in parameters:
        if(parameters[param][1] == "int"):
            if(not parameters[param][0].isdigit()):
                raise Exception("non-digit int detected: " + parameters[param][0])
                return -1
            if(param == "year" and (int(parameters[param][0]) < 2019 or int(parameters[param][0]) >= int(datetime.now().year))):
                raise Exception("invalid year selected: please choose a year in [2019, current)!")
                return -1
        if(parameters[param][1] == "bool"):
            if(not parameters[param][0] in ["True", "False"]):
                raise Exception("invalid boolean detected: " + parameters[param][0])
                return -1
    return 0

In [36]:
def convert_parameters(parameters):
    """
    parameters -- a list-like of tuples: (parameter value, parameter type).

    returns -- parameters, the same list-like as the input, except for each tuple (parameter value, parameter type), parameter value is an object of the type specified by paramter value (leaves it as a string if left unspecified).
    """
    for param in parameters:
        if(parameters[param][1] == "bool"):
            if(parameters[param][0] == "True"):
                parameters[param][0] = True
            if(parameters[param][0] == "False"):
                parameters[param][0] = False
        if(parameters[param][1] == "int"):
            parameters[param][0] = int(parameters[param][0])
    return parameters

In [37]:
# get parameters and make sure they are valid
config = open('/content/drive/MyDrive/SVED Economic Profile Auto-Updaters/Updaters/Census_updater_config.txt', "r")
config = config.read()
parameters = get_parameters(config, ["key", "year", "include_DP05", "include_DP04", "include_DP03"], ["str", "str", "bool", "bool", "bool"])
validate_parameters(parameters)

0

In [38]:
# assign parameters to our varaibles
parameters = convert_parameters(parameters)
KEY = str(parameters["key"][0])
YEAR = str(parameters["year"][0])
include_DP05 = parameters["include_DP05"][0]
include_DP04 = parameters["include_DP04"][0]
include_DP03 = parameters["include_DP03"][0]

In [39]:
print("KEY: ", KEY)
print("YEAR: ", YEAR)

KEY:  356157f0675bf8ad17969b216569bd70244f5d63
YEAR:  2017


# 2. Functions for Querying

## 2.1 Data Getters

In [40]:
def all_number_entries(df : pd.DataFrame()) -> bool:
    numeric_checker = pd.DataFrame()
    for i in range(0, len(locations)):
        for col in df.columns:
            numeric_checker.loc[locations[i],col] = int(isnumber(df.loc[locations[i],col]))
    if ((len(df.columns) * len(df.index)) > numeric_checker.sum().sum()):
        return False
    return True

def flatten_string(s):
    container = []
    temp = ""
    for char in s:
        if(char != "[" and char != "]"):
            if(char == ","):
                container += [temp]
                temp = ""
            else:
                 if(char != '"'):
                        temp += char
    return container

def isnumber(s : str) -> bool:
    has_decimal = False
    for char in s:
        if(not char.isdigit()):
            if (char != "."):
                return False
            else:
                if(has_decimal):
                    return False
                has_decimal = True
    return True

In [41]:
def get_census_table(url : str, col_names : list, index_cols : list = [0], index_names : list = [None]) -> pd.DataFrame:
    """
        url: string containing the url to be parsed for data.
        col_names: list-like of the names that will be used to rename the dataframe.
                    must match the dimensions of the data retrieved.
        index_cols: a list-like of integers containing the indices of the columns which will become the index.
                    constructs a multi-index with the first integer in index_cols being the highest level of
                    the index.
        index_names: a list-like containing the labels for the (multi-leveled) index
        returns: a pandas dataframe containing the data from url.
    """
    # get the data
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    df = pd.DataFrame(ast.literal_eval(str(soup)))

    # set the new index
    new_index = pd.MultiIndex.from_frame(df.iloc[:,index_cols])
    df = df.set_index(new_index)

    # drop columns we are using as the index
    drop_cols = []
    for col_index in index_cols:
        drop_cols += [df.columns[col_index]]
    df = df.drop(columns = drop_cols)

    # change the column names as desired
    if(len(col_names) == len(df.columns)):
        df.columns = col_names

    # get rid of index name
    df.index.names = index_names

    # get rid of the row that contained the column names
    df = df.drop(index = df.iloc[0].name, errors = "ignore")
    return df

In [42]:
def get_custom_tables(SUMMARY : pd.DataFrame(), category_names : list, category_target_names : list, category_target_codes : list) -> list:
    tables = []
    for i in range(0, len(category_names)):
        tables += [pd.DataFrame()]
    for i in range(0, len(category_names)):
        for j in range(0, len(locations)):
            for k in range(0, len(category_target_names[i])):
                tables[i].loc[locations[j], category_target_names[i][k]] = SUMMARY.loc[locations[j],category_target_codes[i][k]]

        # look for non-numbers (incorrect values which make it impossible to enter the data)
        if(not all_number_entries(tables[i])):
                print("ERROR: non-numeric values detected in " + category_names[i])

        # turn to numbers
        for col in tables[i].columns:
            tables[i][col] = tables[i][col].apply(float)
    return tables

In [43]:
def make_summary_table(YEAR : str, locations : list, location_codes : list, group : str) -> pd.DataFrame():
    print("\nQUERYING: " + group + ", YEAR: " + YEAR)
    location_data = []
    for i in range(0, len(locations)):
        # get the data for the location
        if(locations[i] == "BLAINE"):
            url = "https://api.census.gov/data/" + YEAR + "/acs/acs5/profile?get=" + "group(" + group + ")&for=county:" + str(location_codes[i].iloc[1]) + "&in=state:" + str(location_codes[i].iloc[0]) + "&key=" + KEY
        else:
            url = "https://api.census.gov/data/" + YEAR + "/acs/acs5/profile?get=" + "group(" + group + ")&for=place:" + str(location_codes[i].iloc[1]) + "&in=state:" + str(location_codes[i].iloc[0]) + "&key=" + KEY
        # get the raw html and process it
        page = requests.get(url)

        soup = BeautifulSoup(page.content, "html.parser")
        soup = str(soup).replace("null", '"null"') # necessary for ast.literal_eval to work (treats null as a string, "null", instead of a datatype)
        soup = soup.replace("\n", "") # get rid of an extra endline character in the middle of the soup (not necessary)
        df = pd.DataFrame(ast.literal_eval(soup)) # turn the html into a dataframe
        df.columns = df.loc[0,:] # rename the dataframe to be the ACS variable names
        df = df.drop(index = 0)
        location_data += [df]
        print("\tPROCESSED: " + locations[i])

    # make a huge sumamry dataframe
    SUMMARY = pd.DataFrame(columns = location_data[0].columns)
    for ldf in location_data:
        SUMMARY = pd.concat([SUMMARY, ldf], axis = 0)
    SUMMARY.index = locations

    return SUMMARY

## 2.2 Census Code Getters

In [44]:
def filter_keywords_and(kws, s, keep_matches):
    sl = s.lower()
    for kw in kws:
        if kw.lower() not in sl:
            return not keep_matches
    return keep_matches

def filter_keywords_or(kws, s, keep_matches):
    sl = s.lower()
    for kw in kws:
        if kw.lower() in sl:
            return keep_matches
    return not keep_matches

def get_race_codes(year):
    # get variables list for the desired year and reformat
    url = "https://api.census.gov/data/" + str(year) + "/acs/acs5/profile/variables"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    soup = str(soup).replace("null", '"null"') # necessary for ast.literal_eval to work (treats null as a string, "null", instead of a datatype)
    soup = soup.replace("\n", "") # get rid of an extra endline character in the middle of the soup (not necessary)
    df = pd.DataFrame(ast.literal_eval(soup))
    df = df.rename(columns = {0:"name", 1:"label", 2:"concept"})
    df.drop(index = 0)
    df = df.reset_index().drop(columns = ["index"])

    # filter to DP05
    df["mask"] = df["name"].apply(lambda s: "DP05" in s)
    df = df.query("mask == True").drop(columns = ["mask"])

    # get codes
    labels = {}
    # white
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["white", "alone", "not"], s, True) and filter_keywords_and(["percent"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["White"] = label_df.iloc[0,0]
    # latino
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["any"], s, True) and filter_keywords_and(["percent"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    label_df = label_df.sort_values(by = ["name"]) # choose the first one (which will be the "all" category")
    labels["Latino"] = label_df.iloc[0,0]
    # black
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["alone", "not"], s, True) and filter_keywords_or(["black", "african american"], s, True) and filter_keywords_and(["percent"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Black"] = label_df.iloc[0,0]
    # american indian
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["alone", "not"], s, True) and (filter_keywords_and(["indian"], s, True) or filter_keywords_and(["native", "american"],s, True)) and filter_keywords_and(["percent"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["American Indian"] = label_df.iloc[0,0]
    # asian
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["asian", "alone", "not"], s, True) and filter_keywords_and(["percent"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Asian"] = label_df.iloc[0,0]
    # hawaiian/pi
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["hawaiian", "alone", "not"], s, True) and filter_keywords_and(["percent"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Hawaiian/PI"] = label_df.iloc[0,0]
    # other
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["other", "alone", "not"], s, True) and filter_keywords_or(["percent", "hawaiian", "pacific"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Other"] = label_df.iloc[0,0]
    # two or more
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["two", "not"], s, True) and filter_keywords_or(["percent", "other"], s, False) and filter_keywords_or(["hispanic", "latino"], s, True))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Two or More"] = label_df.iloc[0,0]

    return labels

def get_age_codes(year):
    # get variables list for the desired year and reformat
    url = "https://api.census.gov/data/" + str(year) + "/acs/acs5/profile/variables"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    soup = str(soup).replace("null", '"null"') # necessary for ast.literal_eval to work (treats null as a string, "null", instead of a datatype)
    soup = soup.replace("\n", "") # get rid of an extra endline character in the middle of the soup (not necessary)
    df = pd.DataFrame(ast.literal_eval(soup))
    df = df.rename(columns = {0:"name", 1:"label", 2:"concept"})
    df.drop(index = 0)
    df = df.reset_index().drop(columns = ["index"])

    # filter to DP05
    df["mask"] = df["name"].apply(lambda s: "DP05" in s)
    df = df.query("mask == True").drop(columns = ["mask"])

    # get codes
    labels = {}
    # <5 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 5 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["<5 yrs"] = label_df.iloc[0,0]

    # 5 - 9 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 9 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["5-9 yrs"] = label_df.iloc[0,0]

    # 10 - 14 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 14 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["10-14 yrs"] = label_df.iloc[0,0]

    # 15 - 19 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 19 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["15-19 yrs"] = label_df.iloc[0,0]

    # 20 - 24 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 24 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["20-24 yrs"] = label_df.iloc[0,0]

    # 25 - 34 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 34 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["25-34 yrs"] = label_df.iloc[0,0]

    # 35 - 44 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 44 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["35-44 yrs"] = label_df.iloc[0,0]

    # 45 - 54 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 54 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["45-54 yrs"] = label_df.iloc[0,0]

    # 55 - 59 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 59 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["55-59 yrs"] = label_df.iloc[0,0]

    # 60 - 64 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 64 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["60-64 yrs"] = label_df.iloc[0,0]

    # 65 - 74 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 74 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["65-74 yrs"] = label_df.iloc[0,0]

    # 75 - 84 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and([" 84 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["75-84 yrs"] = label_df.iloc[0,0]

    # >85 yrs
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["85 years"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels[">85 yrs"] = label_df.iloc[0,0]

    # Median
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["median", "age"], s, True) and filter_keywords_and(["percent"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Median Age"] = label_df.iloc[0,0]

    return labels

def get_sex_codes(year):
    # get variables list for the desired year and reformat
    url = "https://api.census.gov/data/" + str(year) + "/acs/acs5/profile/variables"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    soup = str(soup).replace("null", '"null"') # necessary for ast.literal_eval to work (treats null as a string, "null", instead of a datatype)
    soup = soup.replace("\n", "") # get rid of an extra endline character in the middle of the soup (not necessary)
    df = pd.DataFrame(ast.literal_eval(soup))
    df = df.rename(columns = {0:"name", 1:"label", 2:"concept"})
    df.drop(index = 0)
    df = df.reset_index().drop(columns = ["index"])

    # filter to DP05
    df["mask"] = df["name"].apply(lambda s: "DP05" in s)
    df = df.query("mask == True").drop(columns = ["mask"])

    # get labels
    labels = {}

    # total
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["sex", "age", "total", "population"], s, True) and filter_keywords_or(["percent", "male", "female", "other", "ratio"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    label_df = label_df.sort_values(by = "name")
    labels["Total Population"] = label_df.iloc[0,0]

    # total male
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["sex", "age", "male"], s, True) and filter_keywords_or(["percent", "female", "other", "ratio"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    label_df = label_df.sort_values(by = "name")
    labels["Total Male"] = label_df.iloc[0,0]

    # total female
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["sex", "age", "female"], s, True) and filter_keywords_or(["percent", "other", "ratio"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    label_df = label_df.sort_values(by = "name")
    labels["Total Female"] = label_df.iloc[0,0]

    return labels

def get_housing_codes(year):
    # get variables list for the desired year and reformat
    url = "https://api.census.gov/data/" + str(year) + "/acs/acs5/profile/variables"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    soup = str(soup).replace("null", '"null"') # necessary for ast.literal_eval to work (treats null as a string, "null", instead of a datatype)
    soup = soup.replace("\n", "") # get rid of an extra endline character in the middle of the soup (not necessary)
    df = pd.DataFrame(ast.literal_eval(soup))
    df = df.rename(columns = {0:"name", 1:"label", 2:"concept"})
    df.drop(index = 0)
    df = df.reset_index().drop(columns = ["index"])

    # filter to DP04
    df["mask"] = df["name"].apply(lambda s: "DP04" in s)
    df = df.query("mask == True").drop(columns = ["mask"])

    # get labels
    labels = {}

    # owner occupied
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["owner", "occupied"], s, True) and filter_keywords_or(["percent", "household", "size"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Owner Occupied"] = label_df.iloc[0,0]

    # renter occupied
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["renter", "occupied"], s, True) and filter_keywords_or(["percent", "household", "size"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Renter Occupied"] = label_df.iloc[0,0]

    # vacant/seasonal occupied
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["vacant", "housing"], s, True) and filter_keywords_or(["percent", "household", "size"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Season/Vac"] = label_df.iloc[0,0]

    return labels

def get_income_codes(year):
       # get variables list for the desired year and reformat
    url = "https://api.census.gov/data/" + str(year) + "/acs/acs5/profile/variables"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    soup = str(soup).replace("null", '"null"') # necessary for ast.literal_eval to work (treats null as a string, "null", instead of a datatype)
    soup = soup.replace("\n", "") # get rid of an extra endline character in the middle of the soup (not necessary)
    df = pd.DataFrame(ast.literal_eval(soup))
    df = df.rename(columns = {0:"name", 1:"label", 2:"concept"})
    df.drop(index = 0)
    df = df.reset_index().drop(columns = ["index"])

    # filter to DP03
    df["mask"] = df["name"].apply(lambda s: "DP03" in s)
    df = df.query("mask == True").drop(columns = ["mask"])

    # get labels
    labels = {}

    # <10k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["than $10,000", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["<10k"] = label_df.iloc[0,0]

    # 10k - 14k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $14,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["10k-14k"] = label_df.iloc[0,0]

    # 15k - 25k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $24,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["15k-25k"] = label_df.iloc[0,0]

    # 25k - 35k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $34,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["25k-35k"] = label_df.iloc[0,0]

    # 35k - 50k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $49,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["35k-50k"] = label_df.iloc[0,0]

    # 50k - 75k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $74,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["50k-75k"] = label_df.iloc[0,0]

    # 75k - 100k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $99,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["75k-100k"] = label_df.iloc[0,0]

    # 100k - 150k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $149,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["100k-150k"] = label_df.iloc[0,0]

    # 150k - 200k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["to $199,999", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["150k-200k"] = label_df.iloc[0,0]

    # >200k
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["$200,000", "more"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels[">=200k"] = label_df.iloc[0,0]

    # Households
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["total", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family", "$"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    label_df = label_df.sort_values(by = "name")
    labels["Total Households"] = label_df.iloc[0,0]

    # Median Income
    df["mask"] = df["label"].apply(lambda s: filter_keywords_and(["median", "household"], s, True) and filter_keywords_or(["benefits", "income"], s, True) and filter_keywords_or(["percent", "family", "$"], s, False))
    label_df = df.query("mask == True").reset_index().drop(columns = ["index"])
    labels["Median Income"] = label_df.iloc[0,0]

    return labels

# 3. Functions for Exporting Data

# 4. Location Codes

In [45]:
temp = YEAR
YEAR = "2022" # can be any recent year, since we are using area codes that don't change year-to-year

# url's for the census website containing area codes
CITY_NAMES_URL = "https://api.census.gov/data/" + YEAR + "/acs/acs5/profile?get=NAME&for=place:*&key=" + KEY
COUNTY_NAMES_URL = "https://api.census.gov/data/" + YEAR + "/acs/acs5/profile?get=NAME&for=county:*&in=state:*&key=" + KEY

# get city names table
city_codes = get_census_table(url = CITY_NAMES_URL, col_names = ["state_code", "city_code"], index_cols = [0], index_names = ["city_name"])
# get county names table
county_codes = get_census_table(url = COUNTY_NAMES_URL, col_names = ["state_code", "county_code"], index_cols = [0], index_names = ["county_name"])

# get codes for each location
HAILEY_CODE = city_codes.loc["Hailey city, Idaho"]
KETCHUM_CODE = city_codes.loc["Ketchum city, Idaho"]
CAREY_CODE = city_codes.loc["Carey city, Idaho"]
BELLEVUE_CODE = city_codes.loc["Bellevue city, Idaho"]
SV_CODE = city_codes.loc["Sun Valley city, Idaho"]
BLAINE_CODE = county_codes.loc["Blaine County, Idaho"]
# reset the year back
YEAR = temp

# 5. Querying Data

## 5.1. Census Profiles

In [46]:
groups = []
if(include_DP05):
    groups += ["DP05"]
if(include_DP04):
    groups += ["DP04"]
if(include_DP03):
    groups += ["DP03"]

census_tables = []
YEAR = YEAR
locations = ["HAILEY", "KETCHUM", "BELLEVUE", "CAREY", "BLAINE", "SV"]
location_codes= [HAILEY_CODE, KETCHUM_CODE, BELLEVUE_CODE, CAREY_CODE, BLAINE_CODE, SV_CODE]
for i in range(0, len(groups)):
    census_tables += [make_summary_table(YEAR, locations, location_codes, groups[i])]


QUERYING: DP05, YEAR: 2017
	PROCESSED: HAILEY
	PROCESSED: KETCHUM
	PROCESSED: BELLEVUE
	PROCESSED: CAREY
	PROCESSED: BLAINE
	PROCESSED: SV

QUERYING: DP04, YEAR: 2017
	PROCESSED: HAILEY
	PROCESSED: KETCHUM
	PROCESSED: BELLEVUE
	PROCESSED: CAREY
	PROCESSED: BLAINE
	PROCESSED: SV

QUERYING: DP03, YEAR: 2017
	PROCESSED: HAILEY
	PROCESSED: KETCHUM
	PROCESSED: BELLEVUE
	PROCESSED: CAREY
	PROCESSED: BLAINE
	PROCESSED: SV


## 5.2. DP05

### 5.2.1. Get Data for Demographics Master

In [47]:
ct_index = 0
include_DP05 = True
if(include_DP05):
    category_names = ["SEX", "AGE", "RACE"]
    category_target_names = [
        ["Total Population","Total Male", "Total Female"],
        ["<5 yrs", "5-9 yrs", "10-14 yrs", "15-19 yrs", "20-24 yrs", "25-34 yrs", "35-44 yrs", "45-54 yrs", "55-59 yrs", "60-64 yrs", "65-74 yrs", "75-84 yrs", ">85 yrs", "Median Age"],
        ["White", "Latino", "Black", "American Indian", "Asian", "Hawaiian/PI", "Other", "Two or More"]
    ]
    category_target_codes_sex_dict = get_sex_codes(YEAR)
    category_target_codes_age_dict = get_age_codes(YEAR)
    category_target_codes_race_dict = get_race_codes(YEAR)

    category_target_codes = [[category_target_codes_sex_dict[label] for label in category_target_names[0]],
                             [category_target_codes_age_dict[label] for label in category_target_names[1]],
                             [category_target_codes_race_dict[label] for label in category_target_names[2]]
                            ]
    DP05_tables = get_custom_tables(census_tables[ct_index], category_names, category_target_names, category_target_codes)
    ct_index += 1

### 5.2.2. Reformat Data for Master

In [48]:
if(include_DP05):
    # sex: none needed!
    i = 0

    # age
    i = 1
    # group by master sheet categories
    DP05_tables[i].loc[:,"<20 yrs"] = DP05_tables[i].loc[:, ["<5 yrs", "5-9 yrs", "10-14 yrs", "15-19 yrs"]].sum(axis = 1)
    DP05_tables[i].loc[:,"20-34 yrs"] = DP05_tables[i].loc[:, ["20-24 yrs", "25-34 yrs"]].sum(axis = 1)
    DP05_tables[i].loc[:,"35-54 yrs"] = DP05_tables[i].loc[:, ["35-44 yrs", "45-54 yrs"]].sum(axis = 1)
    DP05_tables[i].loc[:,"55-64 yrs"] = DP05_tables[i].loc[:, ["55-59 yrs", "60-64 yrs"]].sum(axis = 1)
    DP05_tables[i].loc[:,">64 yrs"] = DP05_tables[i].loc[:, ["65-74 yrs", "75-84 yrs", ">85 yrs"]].sum(axis = 1)
    temp = DP05_tables[i].loc[:,["<20 yrs", "20-34 yrs", "35-54 yrs", "55-64 yrs", ">64 yrs"]].copy(deep = True)
    temp["Total"] = temp.loc[:,:].sum(axis = 1)
    temp["Median Age"] = DP05_tables[i]["Median Age"]
    DP05_tables[i] = temp

    # race
    i = 2
    # group by master sheet categories
    DP05_tables[i].loc[:,"Other"] = DP05_tables[i].loc[:,["Black", "American Indian", "Asian", "Hawaiian/PI", "Other", "Two or More"]].sum(axis = 1)
    DP05_tables[i] = DP05_tables[i].drop(columns = ["Black", "American Indian", "Asian", "Hawaiian/PI", "Two or More"])
    DP05_tables[i]["Total"] = DP05_tables[i].loc[:,:].sum(axis = 1)

## 5.3. DP04

### 5.3.1. Get Data for Housing Stock Master

In [49]:
if(include_DP04):
    category_names = ["UTILIZATION"]
    category_target_names = [
        ["Owner Occupied", "Renter Occupied", "Season/Vac"]
    ]
    category_target_codes_dict = get_housing_codes(YEAR)
    category_target_codes = [[category_target_codes_dict[label] for label in category_target_names[0]]]

    DP04_tables = get_custom_tables(census_tables[ct_index], category_names, category_target_names, category_target_codes)
    ct_index += 1

### 5.3.2. Reformat Data for Master

In [50]:
if(include_DP04):
    # add total column
    DP04_tables[0].loc[:, "Total"] = DP04_tables[0].loc[:,:].sum(axis = 1)

## 5.4 DP03

### 5.4.1. Get Data for Income Master

In [51]:
if(include_DP03):
    category_names = ["INCOME"]
    category_target_names = [
        ["<10k", "10k-14k", "15k-25k", "25k-35k", "35k-50k", "50k-75k", "75k-100k", "100k-150k", "150k-200k", ">=200k", "Total Households", "Median Income"]
    ]
    category_target_codes_dict = get_income_codes(YEAR)
    category_target_codes = [[category_target_codes_dict[label] for label in category_target_names[0]]]
    DP03_tables = get_custom_tables(census_tables[ct_index], category_names, category_target_names, category_target_codes)
    ct_index ++ 1

### 5.4.2 Reformat Data for Master

In [52]:
if(include_DP03):
    df2 = pd.DataFrame()
    # aggregate columns
    df2.loc[:,"<25k"] = DP03_tables[0].loc[:,["<10k", "10k-14k", "15k-25k"]].sum(axis = 1)
    df2.loc[:,"25k-50k"] = DP03_tables[0].loc[:,["25k-35k", "35k-50k"]].sum(axis = 1)
    df2.loc[:, "50k-75k"] = DP03_tables[0].loc[:,"50k-75k"]
    df2.loc[:,">=75k"] = DP03_tables[0].loc[:,["75k-100k", "100k-150k", "150k-200k", ">=200k"]].sum(axis = 1)
    df2.loc[:,"Total Households"] = DP03_tables[0].loc[:,"Total Households"]
    df2.loc[:,"Median Income"] = DP03_tables[0].loc[:,"Median Income"]
    DP03_tables[0] = df2

# 6. Exporting to Master Sheet

## 6.1. Helper Functions

In [53]:
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import Alignment

In [54]:
from openpyxl.utils import get_column_letter
def grid(col : int, row : int) -> str:
    return get_column_letter(col) + str(row)

def year_to_int(year : str) -> int:
    container = ""
    for char in year:
        if char.isdigit():
            container += char
    if(len(container) == 0):
        return -1
    if(len(container) == 2):
        if(int(container) > 65):
            return int("19" + container)
        else:
            return int("20" + container)
    else:
        return int(container)

In [55]:
def insert_into_master(table : pd.DataFrame, table_name : str, col_order : list, source: str, destination : str):
    # open sumamry tab of Demographics Master Sheet
    wb = load_workbook(filename = source)
    wb.active = wb["Summary"]
    sheet = wb.active

    # find the start of the desired table
    curr_row = 1
    while(sheet[grid(1, curr_row)].value != table_name):
        curr_row += 1
    # find the start of the data
    curr_row += 2

    # find where to insert the new data
    while True:
        # find where the new data belongs
        if((sheet[grid(1, curr_row)].value == "end_table") or (year_to_int(sheet[grid(1, curr_row)].value) >= int(YEAR))):
            break
        curr_row += 1
    # if the row doesn't exist, insert it in the right spot
    if(year_to_int(sheet[grid(1, curr_row)].value) != int(YEAR)):
        sheet.insert_rows(idx = curr_row)

    # insert new data
    sheet[grid(1, curr_row)].value = "'" + YEAR[2:]
    sheet[grid(1, curr_row)].alignment = Alignment(horizontal = 'right')
    cols_per_location = len(table.columns)
    i = 2
    while(i < 2 + cols_per_location * len(order)):
        location = order[(i - 2)//cols_per_location]
        sheet[grid(i, curr_row)].value = table.loc[location].iloc[(i-2)%cols_per_location]
        #print(location + ": " + str(table.loc[location][(i-2)%cols_per_location]))
        i += 1
    wb.save(destination)

## 6.2. Export Data

In [56]:
if(include_DP05 == True):
    master = "/content/drive/MyDrive/SVED Economic Profile Auto-Updaters/Masters/MASTER City Demographics 1970-current.xlsx"
    tables = DP05_tables
    table_names = ["SEX_SUMMARY", "AGE_SUMMARY", "RACE_SUMMARY"]
    order = ["CAREY", "BELLEVUE", "HAILEY", "KETCHUM", "SV", "BLAINE"]
    for i in range(0,len(table_names)):
        insert_into_master(tables[i], table_names[i], order, master, master)
        print("Finished: " + table_names[i])

Finished: SEX_SUMMARY
Finished: AGE_SUMMARY
Finished: RACE_SUMMARY


In [57]:
if(include_DP04 == True):
    master = '/content/drive/MyDrive/SVED Economic Profile Auto-Updaters/Masters/MASTER City Housing Stock 2011-current.xlsx'
    tables = DP04_tables
    table_names = ["HOUSING_STOCK_SUMMARY"]
    order = ["CAREY", "BELLEVUE", "HAILEY", "KETCHUM", "SV", "BLAINE"]
    for i in range(0, len(tables)):
        insert_into_master(tables[i], table_names[i], order, master, master)
        print("Finished: " + table_names[i])

Finished: HOUSING_STOCK_SUMMARY


In [58]:
if(include_DP03 == True):
    master = '/content/drive/MyDrive/SVED Economic Profile Auto-Updaters/Masters/MASTER City Income Distribution 2011-current.xlsx'
    tables = DP03_tables
    table_names = ["INCOME_SUMMARY"]
    order = ["CAREY", "BELLEVUE", "HAILEY", "KETCHUM", "SV", "BLAINE"]
    for i in range(0, len(tables)):
        insert_into_master(tables[i], table_names[i], order, master, master)
        print("Finished: " + table_names[i])

Finished: INCOME_SUMMARY
